# Simple Alfalfa Example

## Setup
1. [Alfalfa stack deployed](https://github.com/NREL/alfalfa/wiki/Deployment) locally with **at least two workers**
1. Create `small_office.zip` from `models`, `weather`, and `measures` folders and `small_office.osw`

## Notes
1. For Alfalfa v0.1.0, API reads and writes are implemented over the [Haystack API](https://project-haystack.org/doc/docHaystack/HttpApi), particularly the `read` and `pointWrite` endpoints.


In [ ]:
import os
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient

### Create new alfalfa client object
If you are not hosting the alfalfa server yourself replace the `url` with the one of your server without a trailing slash (this is a known bug and will eventially be fixed)

In [ ]:
ac = AlfalfaClient(url='http://localhost')

### Define paths to models to be uploaded

In [ ]:
m1 = './twobldgs/small_office.zip'
m2 = './twobldgs/small_office.zip'

### Upload sites to alfalfa
The `ac.submit` function returns the site_id which is used to interact with that specific site over the API. A simulation is a site.


Sites can be viewed at http://localhost/sites

Metadata generated can be viewed at http://localhost/api/read?filter=point

In [ ]:
m1_id = ac.submit(m1)
m2_id = ac.submit(m2)


In [ ]:
print(m1_id)
print(m2_id)

### Define parameters to run the simulations

In [ ]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2021, 7, 1, 12, 2, 0)
end_dt = datetime.datetime(2021, 7, 3, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": "true",
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

## Start simulations 
Note: one sim runs / worker, so if you have not scaled your local deployment to workers >= 2, the second worker won't have a chance to start and this code block will not complete.

In [ ]:
ac.start(m1_id, **params)
ac.start(m2_id, **params)

### Get the model's input points
Get a list of all of the model's input points and their values

To set an input value use the `ac.set_inputs(site_id, inputs)` function. 
- `site_id` - the id of the site returned by the `ac.submit` function
- `inputs` - a dictionary of input names and the desired values

In [ ]:
print(ac.inputs(m1_id))
print(ac.inputs(m2_id))

### Advance the model
12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.
Model values are exposed as the `curVal` for points and can be viewed at `http://localhost/api/read?filter=point`

In [ ]:
ac.advance([m1_id, m2_id])

### Get model's outputs
Query the outputs of the models as well as their values

In [ ]:
print(ac.outputs(m1_id))
print(ac.outputs(m2_id))

### Stop the simulations

In [ ]:
ac.stop(m1_id)
ac.stop(m2_id)